In [2]:
import mlflow

import pandas as pd
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar

import xgboost as xgb

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import LinearRegression , Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

import pickle

In [3]:
mlflow.set_tracking_uri("http://127.0.0.1:5000") 
mlflow.set_experiment("Electricity Demand Prediction")

2024/08/12 12:51:27 INFO mlflow.tracking.fluent: Experiment with name 'Electricity Demand Prediction' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/1', creation_time=1723467087565, experiment_id='1', last_update_time=1723467087565, lifecycle_stage='active', name='Electricity Demand Prediction', tags={}>

In [4]:
def read_dataframe(filename):
    df = pd.read_csv(filename)
    df['date'] = pd.to_datetime(df['date'])
    df['demand'] = pd.to_numeric(df['demand'], errors='coerce').astype('float')
    df['year'] = df['date'].dt.year
    df['month'] = df['date'].dt.month
    df['day'] = df['date'].dt.day
    df['hr'] = df['date'].dt.hour
    df['day_of_week'] = df['date'].dt.dayofweek  # Monday=0, Sunday=6
    df['is_weekend'] = df['date'].dt.dayofweek >= 5  # True if weekend, False otherwise
    
    holidays = calendar().holidays(start=df['date'].min(), end=df['date'].max())
    df['holiday'] = df['date'].isin(holidays).astype(int)
    #display(df.head())
    #print(df.dtypes)
    
    
    return df

In [5]:
dataset_path = "/workspaces/Electricity-Demand-Prediction/Model Training/Data/dataset.csv"

In [6]:
df = read_dataframe(dataset_path)

In [7]:
# Define features and target
X = df.drop('demand', axis=1)
X = X.drop('date', axis=1)

y = df['demand']

In [8]:
# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [11]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [12]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        mlflow.sklearn.autolog()
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [14]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

/home/codespace/anaconda3/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [12:52:51] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:2140.72874                        
[1]	validation-rmse:1533.57810                        
[2]	validation-rmse:1194.54188                        
[3]	validation-rmse:1027.19750                        
[4]	validation-rmse:955.35719                         
[5]	validation-rmse:928.09344                         
[6]	validation-rmse:919.88131                         
[7]	validation-rmse:918.80049                         
[8]	validation-rmse:920.15497                         
[9]	validation-rmse:921.59087                         
[10]	validation-rmse:922.68830                        
[11]	validation-rmse:923.57728                        
[12]	validation-rmse:924.19989                        
[13]	validation-rmse:924.61898                        
[14]	validation-rmse:924.86555                        
[15]	validation-rmse:925.03078                        
[16]	validation-rmse:925.11134                        
[17]	validation-rmse:925.15213                        
[18]	valid

2024/08/12 12:52:58 INFO mlflow.tracking._tracking_service.client: 🏃 View run thoughtful-stork-448 at: http://127.0.0.1:5000/#/experiments/1/runs/1f3c2520b1874bb0b8a6be14bebd5c03.

2024/08/12 12:52:58 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1.



  2%|▏         | 1/50 [00:07<05:50,  7.15s/trial, best loss: 925.1696790868147]

...

[0]	validation-rmse:2326.41778                                                 
[1]	validation-rmse:1800.36146                                                 
[2]	validation-rmse:1446.94015                                                 
[3]	validation-rmse:1232.20508                                                 
[4]	validation-rmse:1087.99743                                                 
[5]	validation-rmse:1007.71906                                                 
[6]	validation-rmse:940.48837                                                  
[7]	validation-rmse:905.65858                                                  
[8]	validation-rmse:870.50945                                                  
[9]	validation-rmse:841.38119                                                  
[10]	validation-rmse:826.86397                                                 
[11]	validation-rmse:811.79213                                                 
[12]	validation-rmse:800.01723          

/home/codespace/anaconda3/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [13:07:13] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[20]	validation-rmse:727.47535                                                 
[21]	validation-rmse:724.78379                                                 
[22]	validation-rmse:716.87985                                                 
[23]	validation-rmse:712.28361                                                 
[24]	validation-rmse:707.51296                                                 
[25]	validation-rmse:701.76433                                                 
[26]	validation-rmse:697.73762                                                 
[27]	validation-rmse:693.62561                                                 
[28]	validation-rmse:690.60705                                                 
[29]	validation-rmse:686.26698                                                 
[30]	validation-rmse:682.09985                                                 
[31]	validation-rmse:680.26137                                                 
[32]	validation-rmse:674.66916          

2024/08/12 13:07:22 INFO mlflow.tracking._tracking_service.client: 🏃 View run bedecked-kite-549 at: http://127.0.0.1:5000/#/experiments/1/runs/df8d14fb8b5842fcac77a2b2861f7893.

2024/08/12 13:07:22 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1.



100%|██████████| 50/50 [14:31<00:00, 17.43s/trial, best loss: 384.942476058939]
